In [126]:
import numpy as np
import pandas as pd

In [127]:
train = pd.read_csv('Dataset/train_emoji.csv',header=None)
test = pd.read_csv('Dataset/test_emoji.csv',header=None)

In [128]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [129]:
import emoji

In [130]:
# emoji.EMOJI_UNICODE

In [131]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }

In [132]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴


In [133]:
Xtrain = train[0]
Ytrain = train[1]

Xtest = test[0]
Ytest = test[1]


In [134]:
# printing sentences along with emojis
for i in range(10):
    print(Xtrain[i],emoji.emojize(emoji_dictionary[str(Ytrain[i])]))

never talk to me again 😓
I am proud of your achievements 😁
It is the worst day in my life 😓
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😓
congratulations on your acceptance 😁
The assignment is too long  😓
I want to go play ⚾


In [135]:
f = open('glove.6B.50d.txt',encoding='utf8')

In [136]:
embeddings_index = {}
for line in f:
    values = line.split()
    word = values[0]
    coeffs = np.asarray(values[1:],dtype='float32')
    embeddings_index[word] = coeffs
f.close()

In [137]:
emd_dim = embeddings_index['eat'].shape
print(emd_dim)

(50,)


### Converting sentences into vectors

In [138]:
def getOutputEmbeddings(X):
    maxlen = 10
    embedding_output_matrix = np.zeros((X.shape[0],maxlen,50))
    
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        
        for jx in range(len(X[ix])):
            embedding_output_matrix[ix][jx] = embeddings_index[X[ix][jx].lower()]
    return embedding_output_matrix

In [139]:
emb_Xtrain = getOutputEmbeddings(Xtrain)
emb_Xtest = getOutputEmbeddings(Xtest)

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [140]:
from keras.utils import np_utils

In [141]:
Ytrain = np_utils.to_categorical(Ytrain)
Ytest = np_utils.to_categorical(Ytest)


### Define RNN/LSTM Model

In [142]:
from keras.models import Sequential
from keras.layers import *

In [154]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_8 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_9 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_7 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
__________________________________________________

In [159]:
hist = model.fit(emb_Xtrain,Ytrain,epochs=150,batch_size=64,shuffle=True,validation_split=0.2)

Train on 105 samples, validate on 27 samples
Epoch 1/150
105/105 [==============================] - 0s 494us/step - loss: 0.0080 - accuracy: 1.0000 - val_loss: 1.6053 - val_accuracy: 0.7407
Epoch 2/150
105/105 [==============================] - 0s 476us/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 1.6253 - val_accuracy: 0.6667
Epoch 3/150
105/105 [==============================] - 0s 465us/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 1.6433 - val_accuracy: 0.6667
Epoch 4/150
105/105 [==============================] - 0s 465us/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 1.6534 - val_accuracy: 0.6667
Epoch 5/150
105/105 [==============================] - 0s 451us/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 1.6627 - val_accuracy: 0.6667
Epoch 6/150
105/105 [==============================] - 0s 446us/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 1.6726 - val_accuracy: 0.6296
Epoch 7/150
105/105 [==============================] - 0s 456us/step - loss: 0.0044 -

Epoch 57/150
105/105 [==============================] - 0s 446us/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 1.6084 - val_accuracy: 0.7037
Epoch 58/150
105/105 [==============================] - 0s 361us/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 1.5720 - val_accuracy: 0.7037
Epoch 59/150
105/105 [==============================] - 0s 408us/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 1.5232 - val_accuracy: 0.7407
Epoch 60/150
105/105 [==============================] - 0s 427us/step - loss: 0.0056 - accuracy: 1.0000 - val_loss: 1.4463 - val_accuracy: 0.7037
Epoch 61/150
105/105 [==============================] - 0s 390us/step - loss: 0.0042 - accuracy: 1.0000 - val_loss: 1.4694 - val_accuracy: 0.7037
Epoch 62/150
105/105 [==============================] - 0s 427us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 1.5247 - val_accuracy: 0.7407
Epoch 63/150
105/105 [==============================] - 0s 408us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 1.5747 -

105/105 [==============================] - 0s 427us/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 1.6878 - val_accuracy: 0.7407
Epoch 114/150
105/105 [==============================] - 0s 404us/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 1.7093 - val_accuracy: 0.7407
Epoch 115/150
105/105 [==============================] - 0s 418us/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 1.7260 - val_accuracy: 0.7407
Epoch 116/150
105/105 [==============================] - 0s 380us/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 1.7175 - val_accuracy: 0.7407
Epoch 117/150
105/105 [==============================] - 0s 437us/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 1.7093 - val_accuracy: 0.7407
Epoch 118/150
105/105 [==============================] - 0s 418us/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 1.7017 - val_accuracy: 0.7407
Epoch 119/150
105/105 [==============================] - 0s 380us/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 1.6895 - val_ac

In [160]:
 pred = model.predict_classes(emb_Xtest)

In [161]:
print(pred)

[4 3 3 3 2 2 3 2 4 2 1 2 0 3 1 3 2 2 3 4 3 3 4 0 3 3 2 0 1 2 0 1 3 2 0 3 2
 3 4 2 1 0 0 1 2 3 3 2 3 3 3 0 3 2 3 0]


In [162]:
model.evaluate(emb_Xtest,Ytest)

56/56 [==============================] - 0s 249us/step


[1.9737622397286552, 0.5714285969734192]

In [163]:
for i in range(30):
    print(' '.join(Xtest[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Ytest[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😓
😓
he got a raise
😁
😓
she got me a present
❤️
😓
ha ha ha it was so funny
😁
😁
he is a good friend
❤️
😁
I am upset
❤️
😓
We had such a lovely dinner tonight
❤️
😁
where is the food
🍴
🍴
Stop making this joke ha ha ha
😁
😁
where is the ball
⚾
⚾
work is hard
😓
😁
This girl is messing with me
😓
❤️
are you serious ha ha
😁
😓
Let us go play baseball
⚾
⚾
This stupid grader is not working
😓
😓
work is horrible
😓
😁
Congratulation for having a baby
😁
😁
stop messing around
😓
😓
any suggestions for dinner
🍴
🍴
I love taking breaks
❤️
😓
you brighten my day
😁
😓
I boiled rice
🍴
🍴
she is a bully
😓
❤️
Why are you feeling bad
😓
😓
I am upset
😓
😓
I worked during my birthday
😓
😁
My grandmother is the love of my life
❤️
❤️
enjoy your break
😁
⚾
valentine day is near
❤️
😁
